In [1]:
import numpy as np

In [2]:
training_data = np.load(f'../fashion_train.npy')
test_data = np.load(f'../fashion_test.npy')

np.unique(training_data[:,-1])

array([0, 1, 2, 3, 4], dtype=uint8)

In [39]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def feedforward(data, num_hidden_layers, learning_rate, epochs):
    num_samples, num_features = data.shape[0], data.shape[1]-1
    input_layer_size = num_features
    hidden_layer_size = num_features

    input_layer_weights = np.random.randn(input_layer_size, hidden_layer_size)
    input_layer_bias = np.zeros((1, input_layer_size))

    hidden_layer_weights = {f'n{i}': np.random.randn(hidden_layer_size, hidden_layer_size) for i in range(num_hidden_layers)}
    hidden_layer_bias = {f'n{i}': np.zeros((1, hidden_layer_size)) for i in range(num_hidden_layers)}

    output_layer_size = len(np.unique(data[:,-1]))
    output_layer_weights = np.random.randn(hidden_layer_size, output_layer_size)
    output_layer_bias = np.zeros((1, output_layer_size))


    hidden_layer_inputs = {}
    hidden_layer_outputs = {}

    for i in range(num_hidden_layers):
        if i == 0:
            hidden_layer_inputs = {f'n{i}': np.dot(data[:,:-1], input_layer_weights) + input_layer_bias}
        else:
            hidden_layer_inputs = {f'n{i}': np.dot(hidden_layer_inputs[f'n{i-1}'], hidden_layer_weights[f'n{i-1}']) + hidden_layer_bias[f'n{i-1}']}

        hidden_layer_outputs = {f'n{i}': sigmoid(hidden_layer_inputs[f'n{i}'])}

    output_layer_input = np.dot(hidden_layer_outputs[f'n{num_hidden_layers-1}'], output_layer_weights) + output_layer_bias

    output_layer_output = sigmoid(output_layer_input)

    return output_layer_output


feedforward(training_data, 4, 0.001,100)

C:\Users\krist\AppData\Local\Temp\ipykernel_37120\2299662063.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


array([[9.99999999e-01, 9.09707806e-01, 1.56422036e-07, 6.09490570e-11,
        1.00000000e+00],
       [9.99999965e-01, 4.72617150e-12, 7.51891777e-07, 3.89294956e-02,
        1.00000000e+00],
       [1.00000000e+00, 9.93720395e-01, 2.16356482e-09, 3.62958780e-05,
        1.00000000e+00],
       ...,
       [9.99992783e-01, 6.34589593e-08, 1.04512838e-10, 1.27716690e-06,
        1.00000000e+00],
       [9.99999997e-01, 9.99129648e-01, 7.69095624e-09, 1.50990375e-01,
        1.00000000e+00],
       [1.00000000e+00, 7.11705530e-10, 3.73481667e-13, 2.38260334e-10,
        1.00000000e+00]])

In [42]:
def cost_function(y_pred, y):
    return np.mean([(y_i - y_pred_i)**2 for y_i, y_pred_i in zip(y, y_pred)])

def predict(beta_0, beta_1, x):
    return [beta_0 + beta_1 * x_i for x_i in x]


def backpropagation(data, num_hidden_layers, learning_rate, epochs):
    
    ff = feedforward(data, num_hidden_layers, learning_rate, epochs)
    loss = cost_function(ff, data[:,-1])

    return loss


backpropagation(training_data, 4, 0.001,100)

C:\Users\krist\AppData\Local\Temp\ipykernel_37120\2299662063.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


4.254511987211752